In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# Formulação FEM para Navier-Stokes

No notebook anterior tratamos EDPs de diferentes tipos, todos lineares. Mas a mema abordagem utilizada não funciona para o problema que trataremos aqui: resolver a Equação de Navier Stokes.

Antes disso, teremos as quantidades:

* Velocidade do fluido: $\mathbf{u} = \left(u_1, \ u_2, \ u_3\right) = \left(u, \ v, \ w\right)$
* Pressão do fluido: $p$
* Fator de campo: $\mathbf{g}$
* Densidade: $\rho$
* Fluido newtoniano de viscosidade: $\mu$
* Tempo: $t$
* Espaço: $\left(x, \ y, \ z\right) = \left(x_1, \ x_2, \ x_3\right)$

Resolver a equação de Navier Stokes se resume a encontrar os valores da velocidade $\mathbf{u}$ e pressão $p$ ao longo do domínio $\Omega_{xyzt}$.

Na notação vetorial temos a equação de balanço de massa dado por $(1.1)$ e o balanço de momento dado por $(1.2)$

$$\dfrac{\partial \rho}{\partial t} + \nabla \cdot \left(\rho \mathbf{u}\right) = 0 \tag{1.1}$$

$$\dfrac{\partial \left(\rho \mathbf{u}\right)}{\partial t} + \nabla \cdot \left( \rho \mathbf{u} \otimes \mathbf{u} \right) = -\nabla p + \rho \mathbf{g} + \mu \nabla^2 \mathbf{u}\tag{1.2}$$


Na notação indicial, $(1.1)$ e $(1.2)$ se tornam $(1.3)$ e $(1.4)$

$$\dfrac{\partial \rho}{\partial t} + \sum_{i = 1}^{3} \dfrac{\partial \left(\rho u_i\right)}{\partial x_i} = 0
\tag{1.3}$$
$$\dfrac{\partial \rho u_i}{\partial t} + \sum_{j=1}^{3} \dfrac{\partial \left(\rho u_i u_j\right)}{\partial x_j} = -\dfrac{\partial p}{\partial x_i} + \rho g_i + \mu \sum_{j=1}^{3} \dfrac{\partial^2 u_i}{\partial x_j^2} \ \ \ \ \ \forall i \tag{1.4}$$

Para o primeiro exemplo, simplificaremos: Utilizaremos uma malha bidimensional $u_3 = 0$ de um fluido incompressível $\rho = \text{const}$, com campo nulo $\mathbf{g} = \mathbf{0}$.

Também fazemos a adimensionalização do problema, resultando em $(1.5)$, $(1.6)$ e $(1.7)$:

$$
\begin{align*}
\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y} & = 0 \tag{1.5} \\ \dfrac{\partial u}{\partial t} + u \dfrac{\partial u}{\partial x} + v\dfrac{\partial u}{\partial y} & = -\dfrac{\partial p}{\partial x} + \mu \left(\dfrac{\partial^2 u}{\partial x^2}+\dfrac{\partial^2 u}{\partial y^2}\right) \tag{1.6} \\
\dfrac{\partial v}{\partial t} + u\dfrac{\partial v}{\partial x} + v\dfrac{\partial v}{\partial y} & = -\dfrac{\partial p}{\partial y} + \mu \left(\dfrac{\partial^2 v}{\partial x^2}+\dfrac{\partial^2 v}{\partial y^2}\right) \tag{1.7}
\end{align*}
$$

Na malha $\Omega_{xyt} = \left[0, \ 1\right] \times \left[0, \ 1\right] \times \left[0, \ \infty\right)$.

### FEM com BSpline

Com a equação de navier stokes, iremos utilizar BSpline apenas para as variáveis no espaço.
Desta forma, calculamos progressivamente no tempo, resolvendo um sistema para cada passo de tempo e obtendo uma EDO em relação a $t$ .
Então no tempo $t_{l} \in \left[0, \ \infty\right)$

$$\begin{align*}u_{l}(x, \ y) & = \sum_{i=0}^{nx-1} \sum_{j=0}^{ny-1} N_{i,px}(x) \cdot N_{j, py}(y) \cdot U_{l,i,j} \tag{1.8} \\
v_{l}(x, \ y) & = \sum_{i=0}^{nx-1} \sum_{j=0}^{ny-1} N_{i,px}(x) \cdot N_{j, py}(y) \cdot V_{l,i,j} \tag{1.9} \\ p_{l}(x, \ y) & = \sum_{i=0}^{nx-1} \sum_{j=0}^{ny-1} N_{i,px}(x) \cdot N_{j, py}(y) \cdot P_{l,i,j} \tag{1.10}\end{align*}$$

Então no tempo $t_l$, há $3n_xn_y$ variáveis a serem encontradas.
A partir daqui omitiremos o indice $l$.

Na formulação de Galerkin, teremos para a Equação $(1.5)$:

$$\int_{\Omega} \left(\dfrac{\partial u}{\partial x} + \dfrac{\partial u}{\partial y}\right)\cdot \varphi \ d\Omega = 0 \ \ \ \ \ \forall \varphi$$
$$\begin{align*}\sum_{a=0}^{nx-1} \sum_{b=0}^{ny-1} & \left(\int_{0}^{1} N_{i,px} \dfrac{dN_{a,px}}{dx} \ dx\right)\left(\int_{0}^{1}N_{j,py}N_{b,py} \ dy \right)U_{a,b} \\ & +\left(\int_{0}^{1} N_{i,px} N_{a,px} \ dx\right)\left(\int_{0}^{1}N_{j,py}\dfrac{dN_{b,py}}{dy} \ dy \right)V_{a,b} = 0 \ \ \ \ \forall i, \ j \end{align*} $$
$$\sum_{a,b}\left(\left[H_{px, px-1}\right] \left[D_{px}\right]^{T}\right)_{ia}\left[H_{py,py}\right]_{jb} \left[U\right]_{ab} + \left[H_{px,px}\right]_{ia}\left(\left[H_{py, py-1}\right] \left[D_{py}\right]^{T}\right)_{jb} \left[V\right]_{ab}= 0 \ \ \ \forall i, \ j$$

Já para a Equação $(1.6)$ teremos

$$\int_{\Omega} \left( \dfrac{\partial u}{\partial t} + u\dfrac{\partial u}{\partial x} + v\dfrac{\partial u}{\partial y} + \dfrac{\partial p}{\partial x} - \mu \left(\dfrac{\partial^2 u}{\partial x^2}+\dfrac{\partial^2 u}{\partial y^2}\right)\right) N_{i,px}N_{j,py} \ d\Omega = 0 \ \ \ \ \forall i, \ j$$
---------------
$$\begin{align*} \sum_{a, b} &  \left(\int_{\Omega} N_{i,px}N_{j,py}N_{a,px}N_{b,py} \ d\Omega\right) \dfrac{dU_{a,b}}{dt} \\ 
+ \sum_{a1, b1, a2, b2} & \left(\int_{\Omega} N_{a1,px}N_{b1,py} \dfrac{dN_{a2,px}}{dx} N_{b2,py} N_{i,px}N_{j,py} \ d\Omega\right) U_{a1,b1} U_{a2,b2} \\
+ \sum_{a_1, b_1, a_2, b_2} & \left(\int_{\Omega} N_{a1,px}N_{b1,py} N_{a2,px} \dfrac{dN_{b2,py}}{dy} N_{i,px}N_{j,py} \ d\Omega\right) V_{a1,b1}U_{a2,b2} \\
+ \sum_{a, b} & \left(\int_{\Omega} N_{i,px}N_{j,py}\dfrac{dN_{a,px}}{dx}N_{b,py} \ d\Omega\right) P_{a,b} \\ 
- \mu \sum_{a,b} &  \left(\int_{\Omega}N_{i,px}N_{j,py}\dfrac{d^2N_{a,px}}{dx^2}N_{b,py} \ d\Omega\right) U_{a,b} \\ 
- \mu \sum_{a,b}&  \left(\int_{\Omega}N_{i,px}N_{j,py}N_{a,px}\dfrac{d^2N_{b,py}}{dy^2} \ d\Omega\right) U_{a,b} = 0 \ \ \ \forall i, \ j\end{align*}$$
-------------
$$\begin{align*} \sum_{a, b} & \left[H_{px,px}\right]_{i,a} \left[H_{py,py}\right]_{j,b} \dfrac{dU_{a,b}}{dt} \\
+ \sum_{a1, b1, a2, b2} & \left(\left[H_{px,px,px-1}\right]\left[D_{px}^{T}\right]\right)_{i,a1,a2} \left[H_{py,py,py}\right]_{j,b1,b2} U_{a1,b1} U_{a2,b2} \\
+ \sum_{a_1, b_1, a_2, b_2} & \left[H_{px,px,px}\right]_{i,a1,a2} \left(\left[H_{py,py,py-1}\right]\left[D_{py}\right]^{T}\right)_{j,b1,b2} V_{a1,b1}U_{a2,b2} \\
+ \sum_{a, b}  & \left(\left[H_{px,px-1}\right]\left[D_{px}\right]^{T}\right)_{i,a} \left[H_{py,py}\right]_{j,b} P_{a,b} \\ 
- \mu \sum_{a,b} &  \left(\left[X_{bound}\right]\left[D_{px}\right]^{T}-\left[D_{px}\right]\left[H_{px-1,px-1}\right]\left[D_{px}\right]^{T}\right)_{i,a}\left[H_{py,py}\right]_{j,b} U_{a,b} \\ 
- \mu \sum_{a,b}& \left[H_{px,px}\right]_{i,a} \left(\left[Y_{bound}\right]\left[D_{py}\right]^{T}-\left[D_{py}\right]\left[H_{py-1,py-1}\right]\left[D_{py}\right]^{T}\right)_{j,b} U_{a,b} = 0 \ \ \ \forall i, \ j\end{align*}$$

Da mesma forma com a equação $(1.7)$ obtemos:


$$\begin{align*} \sum_{a, b} & \left[H_{px,px}\right]_{i,a} \left[H_{py,py}\right]_{j,b} \dfrac{dU_{a,b}}{dt} \\
+ \sum_{a1, b1, a2, b2} & \left(\left[H_{px,px,px-1}\right]\left[D_{px}^{T}\right]\right)_{i,a1,a2} \left[H_{py,py,py}\right]_{j,b1,b2} U_{a1,b1} U_{a2,b2} \\
+ \sum_{a_1, b_1, a_2, b_2} & \left[H_{px,px,px}\right]_{i,a1,a2} \left(\left[H_{py,py,py-1}\right]\left[D_{py}\right]^{T}\right)_{j,b1,b2} V_{a1,b1}U_{a2,b2} \\
+ \sum_{a, b}  & \left(\left[H_{px,px-1}\right]\left[D_{px}\right]^{T}\right)_{i,a} \left[H_{py,py}\right]_{j,b} P_{a,b} \\ 
- \mu \sum_{a,b} &  \left(\left[X_{bound}\right]\left[D_{px}\right]^{T}-\left[D_{px}\right]\left[H_{px-1,px-1}\right]\left[D_{px}\right]^{T}\right)_{i,a}\left[H_{py,py}\right]_{j,b} V_{a,b} \\ 
- \mu \sum_{a,b}& \left[H_{px,px}\right]_{i,a} \left(\left[Y_{bound}\right]\left[D_{py}\right]^{T}-\left[D_{py}\right]\left[H_{py-1,py-1}\right]\left[D_{py}\right]^{T}\right)_{j,b} V_{a,b} = 0 \ \ \ \forall i, \ j\end{align*}$$